In [2]:
import urllib.request as req
from bs4 import BeautifulSoup
import os.path
import time

기상청 주소 https://www.weather.go.kr/weather/lifenindustry/sevice_rss.jsp

In [3]:
url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108'
dt = time.strftime('%Y%m%d',time.localtime(time.time()))
xml_file = 'forecast_{}.xml'.format(dt)

In [ ]:
# xml 파일로 저장
if not os.path.exists(xml_file) : 
    req.urlretrieve(url, xml_file)

In [8]:
xml = open(xml_file, 'r' , encoding='utf-8').read()
soup = BeautifulSoup(xml, 'html.parser')

In [12]:
location = soup.select_one('location')
location.select_one('city').text

'서울'

In [13]:
locations = soup.select('location')

In [17]:
info = dict()
for location in locations : 
    city = location.select_one('city').text
    weather = location.select_one('wf').text
    if not (weather in info):
        info[weather] = []
    info[weather].append(city)

xml은 항목만 잘 찾으면 됨!!

In [18]:
info

{'맑음': ['서울', '인천', '수원', '파주', '이천', '평택'],
 '구름많음': ['춘천', '원주', '강릉', '대전', '세종', '홍성', '청주', '충주', '영동'],
 '흐리고 비': ['광주',
  '목포',
  '여수',
  '순천',
  '광양',
  '나주',
  '전주',
  '군산',
  '정읍',
  '남원',
  '고창',
  '무주',
  '부산',
  '울산',
  '창원',
  '진주',
  '거창',
  '통영',
  '대구',
  '안동',
  '포항',
  '경주',
  '울진',
  '울릉도',
  '제주',
  '서귀포']}

### 생활정보 - 생활지수
케이웨더 http://www.kweather.co.kr/forecast/forecast_living_jisu.html

In [21]:
import requests
from bs4 import BeautifulSoup

In [23]:
url = 'http://www.kweather.co.kr/forecast/forecast_living_jisu.html'
response = requests.get(url)

In [27]:
soup = BeautifulSoup(response.text)
# soup

In [49]:
ul = soup.select_one('#jisu_navi_content')
a_href = ul.select_one('a')['href']
a_href

"javascript:startRequest('../data/JISU/11B00000.xml','1');"

In [53]:
address1 = a_href.split('..')[1]
address2 = address1.split("',")[0]
address2

'/data/JISU/11B00000.xml'

In [54]:
a_href_all = ul.select('a')
xml_list = []
for item in a_href_all:
    a_href = item['href']
    address1 = a_href.split('..')[1]
    address2 = address1.split("',")[0]
    xml_list.append(address2)

In [55]:
xml_list

['/data/JISU/11B00000.xml',
 '/data/JISU/11D10000.xml',
 '/data/JISU/11D20000.xml',
 '/data/JISU/11C10000.xml',
 '/data/JISU/11C20000.xml',
 '/data/JISU/11H10000.xml',
 '/data/JISU/11H20000.xml',
 '/data/JISU/11F10000.xml',
 '/data/JISU/11F20000.xml',
 '/data/JISU/11G00000.xml']

In [56]:
import re

In [58]:
xml_re_list = []
for item in a_href_all:
    a_href = item['href']
    info = re.search('/data/JISU/.+.xml',a_href)
    xml_re_list.append(info.group())


In [59]:
xml_re_list

['/data/JISU/11B00000.xml',
 '/data/JISU/11D10000.xml',
 '/data/JISU/11D20000.xml',
 '/data/JISU/11C10000.xml',
 '/data/JISU/11C20000.xml',
 '/data/JISU/11H10000.xml',
 '/data/JISU/11H20000.xml',
 '/data/JISU/11F10000.xml',
 '/data/JISU/11F20000.xml',
 '/data/JISU/11G00000.xml']

In [60]:
# 서울 / 경기
url = 'http://www.kweather.co.kr/{}'.format(xml_list[0])

In [66]:
response = requests.get(url)
response.status_code
response.encoding = None

In [68]:
soup = BeautifulSoup(response.text)
# soup

In [71]:
jisu_name = soup.select_one('name').text
jisu_jnum = soup.select_one('jnum').text
jisu_jtext = soup.select_one('jtext').text
print(jisu_name,jisu_jnum,jisu_jtext)

나들이 80 더워서 걱정이네요


In [81]:
jisu_name_all = soup.select('name')
jisu_jnum_all = soup.select('jnum')
jisu_jtext_all = soup.select('jtext')
print(len(jisu_name_all))

42


In [84]:
jisu_sk_list = []
for i in range(len(jisu_name_all)):
    jisu_i = (jisu_name_all[i].text,
               jisu_jnum_all[i].text,
               jisu_jtext_all[i].text)
    jisu_sk_list.append(jisu_i)
    
jisu_sk_list

[('나들이', '80', '더워서 걱정이네요'),
 ('빨래', '90', '아주 잘 말라요'),
 ('세차', '90', '세차효과 길어요'),
 ('난방', '20', '난방 불필요해요'),
 ('불조심', '20', '소화기 점검은 평소에'),
 ('운동', '90', '컨디션이 좋습니다'),
 ('감기', '60', '저항력을 높이세요'),
 ('공해', '60', '대체로 좋은 편'),
 ('냉방', '60', '쾌적한 실내온도 유지하세요'),
 ('불쾌', '80', '마음의 폭을 넓히세요'),
 ('자외선', '80', '장시간 야외활동은 위험해요'),
 ('모기', '90', '모기 공격 조심'),
 ('부패', '40', '냉장보관이 안전해요'),
 ('골프', '80', '라운딩에 좋은 조건입니다.'),
 ('나들이', '70', '더위먹지 않도록 주의'),
 ('빨래', '90', '아주 잘 말라요'),
 ('세차', '90', '세차하기 좋은날'),
 ('난방', '20', '난방 불필요해요'),
 ('불조심', '20', '소화기 점검은 평소에'),
 ('운동', '90', '컨디션이 좋습니다'),
 ('감기', '60', '저항력을 높이세요'),
 ('공해', '60', '대체로 좋은 편'),
 ('냉방', '60', '적당한 냉방이 필요해요'),
 ('불쾌', '80', '말다툼은 금물'),
 ('자외선', '90', '장시간 야외활동은 위험해요'),
 ('모기', '40', '모기가 힘이 없어요'),
 ('부패', '60', '냉장고에 보관하세요'),
 ('골프', '80', '라운딩에 좋은 조건입니다.'),
 ('나들이', '70', '더위먹지 않도록 주의'),
 ('빨래', '70', '마르는데 지장 없어요'),
 ('세차', '90', '광택도 내보세요'),
 ('난방', '20', '난방은 필요없어요'),
 ('불조심', '20', '소화기 점검은 평소에'),
 ('운동', '70', '수분공급 충분히'),
 ('감기', 